This transforms the raw csv files (just single column with all the wanted column values in a quasi-uniform pattern to a csv file with the wanted columns. We need to set up checkers to make sure that the rows are correct because the tabula application isnt perfect.

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
r2011 = pd.read_csv("raw_CSV/2011.csv", sep=',', header=None)
r2012 = pd.read_csv("raw_CSV/2012.csv", sep=',', header=None)
r2013 = pd.read_csv("raw_CSV/2013.csv", sep=',', header=None)
r2014 = pd.read_csv("raw_CSV/2014.csv", sep=',', header=None)
r2015 = pd.read_csv("raw_CSV/2015.csv", sep=',', header=None)
r2016 = pd.read_csv("raw_CSV/2016.csv", sep=',', header=None)
r2017 = pd.read_csv("raw_CSV/2017.csv", sep=',', header=None)
r2018 = pd.read_csv("raw_CSV/2018.csv", sep=',', header=None)

In [3]:
r2015

,0,1,2,3,4,5
0,Noise Disturbance,NaN,NaN,NaN,NaN,NaN
1,The Village West Building 1,NaN,NaN,NaN,NaN,NaN
2,Date Reported 9/30/2015,NaN,NaN,NaN,NaN,NaN
3,Incident/Case# 1509300005,NaN,NaN,NaN,NaN,NaN
4,Date Occurred 9/30/2015,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
61591,Incident/Case# 1511230053,NaN,NaN,NaN,NaN,NaN
61592,Date Occurred 11/23/2015,NaN,NaN,NaN,NaN,NaN
61593,Time Occurred 10:33 PM,NaN,NaN,NaN,NaN,NaN
61594,Summary: Male and female heard yelling at each...,NaN,NaN,NaN,NaN,NaN


In [7]:
# we know the pattern is this event -> location -> "date reported" -> Incident/Case# -> Date Occured -> Time Occurred -> Summary -> Dispostion.
# Therefore we can write an algorithm that adds None to all the if the iterator does not match)
def cleaner(raw):
    isIncorrectSyntax = True
    combine = raw.dropna()
    try:
        raw[0][combine.index] = combine[0] + " " + combine[1]
        raw.dropna(subset=[0], inplace=True)
    except:
        pass
    faultyData = np.array(raw[0])
    print(faultyData)
    while isIncorrectSyntax:
        for i in range(len(faultyData)):
            if i % 8 == 0:
                if bool(re.search("Date Reported", faultyData[i])) or bool(re.search("Incident/Case#", faultyData[i])) \
                or bool(re.search("Incident/Case#", faultyData[i])) or bool(re.search("Date Occurred", faultyData[i])) \
                or bool(re.search("Time Occurred", faultyData[i])) or bool(re.search("Summary:", faultyData[i])) \
                or bool(re.search("Disposition:", faultyData[i])):
                    faultyData = np.insert(faultyData, i, "Unknown Event")
            if i % 8 == 1:
                if bool(re.search("Date Reported", faultyData[i])) or bool(re.search("Incident/Case#", faultyData[i])) \
                or bool(re.search("Incident/Case#", faultyData[i])) or bool(re.search("Date Occurred", faultyData[i])) \
                or bool(re.search("Time Occurred", faultyData[i])) or bool(re.search("Summary:", faultyData[i])) \
                or bool(re.search("Disposition:", faultyData[i])):
                    faultyData = np.insert(faultyData, i, "Unknown Location")
            if i % 8 == 2:
                if not bool(re.search("Date Reported", faultyData[i])):
                    print(faultyData[i])
                    faultyData = np.insert(faultyData, i, "Date Reported")
                    break
            if i % 8 == 3:
                if not bool(re.search("Incident/Case#", faultyData[i])):
                    print(faultyData[i])
                    faultyData = np.insert(faultyData, i, "Incident/Case#")
                    break
            if i % 8 == 4:
                if not bool(re.search("Date Occurred", faultyData[i])):
                    print(faultyData[i])
                    faultyData = np.insert(faultyData, i, "Date Occurred")
                    break
            if i % 8 == 5:
                if not bool(re.search("Time Occurred", faultyData[i])):
                    print(faultyData[i])
                    faultyData = np.insert(faultyData, i, "Time Occurred")
                    break
            if i % 8 == 6:
                if not bool(re.search("Summary:", faultyData[i])):
                    print(faultyData[i])
                    faultyData = np.insert(faultyData, i, "Summary:")
                    break
                else:
                    # This is pretty shitty way to check, but better than nothing
                    # another potential checker is [^DELIS] from the first letters of the column headers
                    if bool(re.match(r"^[\$a-z0-9]", faultyData[i + 1])):
                        print(faultyData[i + 1])
                        faultyData[i] = faultyData[i] + " " + faultyData[i + 1]
                        faultyData = np.delete(faultyData, i + 1)
                        break
            if i % 8 == 7:
                if not bool(re.search("Disposition", faultyData[i])):
                    print(faultyData[i])
                    faultyData = np.insert(faultyData, i, "Disposition:")
                    break
        else:
            isIncorrectSyntax = False
    clean = pd.DataFrame(np.array(faultyData).reshape(len(faultyData) // 8, 8))
    nameChange = {0: "Event", 1: "Location", 2:"Date Reported", 3: "Incident/Case#", 4: "Date Occurred", 5:"Time Occurred",
             6: "Summary:", 7: "Disposition:"}
    clean.rename(columns=nameChange, inplace=True)
    clean["Date Reported"] = clean["Date Reported"].map(lambda x: x.replace("Date Reported", "").replace('"', "").strip())
    clean["Date Occurred"] = clean["Date Occurred"].map(lambda x: x.replace("Date Occurred", "").replace('"', "").strip())  
    clean["Incident/Case#"] = clean["Incident/Case#"].map(lambda x: x.replace("Incident/Case#", "").replace('"', "").strip())
    clean["Time Occurred"] = clean["Time Occurred"].map(lambda x: x.replace("Time Occurred", "").replace('"', "").strip())
    clean["Summary:"] = clean["Summary:"].map(lambda x: x.replace("Summary:", "").replace('"', "").strip())
    clean["Disposition:"] = clean["Disposition:"].map(lambda x: x.replace("Disposition:", "").replace("Disposition", "").replace('"', "").strip())
    return clean
    

In [8]:
cleaner(r2015)

<ipython-input-7-aeb5e9e3d17d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw[0][combine.index] = combine[0] + " " + combine[1]
/Users/nicklin/opt/anaconda3/envs/ucsdCrime/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


['Noise Disturbance' 'The Village West Building 1'
 'Date Reported 9/30/2015' ... 'Time Occurred 10:33 PM'
 'Summary: Male and female heard yelling at each other'
 'Disposition: Checks OK']
passenger side windows
taken
Hookah and in possession of alcohol
Disposition: Student Conduct Referral
Disposition: Student Conduct Referral
Disposition: Student Conduct Referral
Disposition: Student Conduct Referral
Disposition: Student Conduct Referral
observed empty alcohol bottles in suite and other bottles with alcohol still inside them, the odor
of cigarette smoke and noticed the smoke detector had been covered with a towel
Citizen Contact
motorist
different locations, loss $1,787.00
Medical Aid
phone number
caring a bag of clothes, was requesting assistance getting to Oceanside
deceased, no signs of foul play involved
Alcohol Contact
Disposition: Service Provided
Disposition: Closed By Citation, Adult
suicide
Officers
Disposition: Transported To Hospital
Disposition: Transported To Hospital
D

KeyboardInterrupt: 

In [38]:
#c2011 = cleaner(r2011)
#c2012 = cleaner(r2012)
#c2013 = cleaner(r2013)
#c2014 = cleaner(r2014)
#c2015 = cleaner(r2015)
#c2016 = cleaner(r2016)
#c2017 = cleaner(r2017)
#c2017 = cleaner(r2018)

I deleted and combined extraneous row here, and checked that tabula got all the cases.  
The following code helped identify where errors were as errors were in locations where case# or dispositions are empty.  

In [2]:
#c2011[c2011.isnull()["Incident/Case#"]]
#c2012[c2012.isnull()["Incident/Case#"]]
#c2013[c2013.isnull()["Incident/Case#"]]
#c2014[c2014.isnull()["Incident/Case#"]]
#c2015[c2015.isnull()["Incident/Case#"]]
#c2016[c2016.isnull()["Incident/Case#"]]
#c2017[c2017.isnull()["Incident/Case#"]]
#c2018[c2018.isnull()["Incident/Case#"]]

In [3]:
#c2011[c2011.isnull()["Disposition:"]]
#c2012[c2012.isnull()["Disposition:"]]
#c2013[c2013.isnull()["Disposition:"]]
#c2014[c2014.isnull()["Disposition:"]]
#c2015[c2015.isnull()["Disposition:"]]
#c2016[c2016.isnull()["Disposition:"]]
#c2017[c2017.isnull()["Disposition:"]]
#c2018[c2018.isnull()["Disposition:"]]

Main reason i didn't combine them is to make processing easier. Finding missing values on the PDF that tabula didn't pick up takes up to 10 seconds for it to search. 

I used excel since it was easier to input
A few disadavantages of using excel:
1. If you mess up in an earlier stage or want to revert back to an earlier stage, you're out of luck
2. Its not reproducable; you're gonna have to rely on me on making sure everything is correct

In [ ]:
#c2011.to_csv("c2011.csv", index=False)
#c2012.to_csv("c2012.csv", index=False)
#c2013.to_csv("c2013.csv", index=False)
#c2014.to_csv("c2014.csv", index=False)
#c2015.to_csv("c2015.csv", index=False)
#c2016.to_csv("c2016.csv", index=False)
#c2017.to_csv("c2017.csv", index=False)
#c2018.to_csv("c2018.csv", index=False)